# Test DataIO Python Library (New) routines

This Jupyter notebook is a revised testing notebook aimed at testing the new CSV importing functions that
manipulate the strings on import to convert them into the expected lists.

In [ ]:
# This forces a reload of any external library file if it changes.  
# Useful when developing external libraries since otherwise Jupyter 
# will not re-import any library without restarting the python kernel.

%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd

# Import COVID data IO routines from external python library
import COVIDlib.data_IO as COVID_IO

## Define variables of interest below
data_dir = 'our_data/'    # Data directory for the COVID datafiles
test_dir = 'test_data/'   # Data directory for storing test datafiles

## Define FIPS corresponding to various local areas
ClayFIPS = 27027
CassFIPS = 38017
MNFIPS = 27
NDFIPS = 38

# Test John Hopkins DataIO

Execute and test the John Hopkins DataIO routines first authored by Luke

In [ ]:
# Trying the new routines to import dataframes from CSV with
# conversion of strings to lists.  Also allows a single 
# getLocalDataFrame() function for pulling to local data from
# ANY of our dataframes (since they all index based on FIPS).

# Retrieve John Hopkins dataframes
(newJH_state_df, newJH_cnty_df) = COVID_IO.CSVtoCDRDataFrames()

# Retrieve Test dataframes
test_cntyfile = test_dir+"TEST_countylevel_combinedCDR.csv"
test_statefile = test_dir+"TEST_statelevel_combinedCDR.csv"
(TESTnewJH_state_df, TESTnewJH_cnty_df) = COVID_IO.CSVtoCDRDataFrames(stateFile = test_statefile, countyFile = test_cntyfile)

# Retrieve State-Level data for Minnesota
newMN_df = COVID_IO.getLocalDataFrame(MNFIPS, newJH_state_df)
# Retrieve County-level data for Clay County
newCLAY_df = COVID_IO.getLocalDataFrame(ClayFIPS, newJH_cnty_df)

# Retrieve TEST State-Level data for Minnesota
TESTnewMN_df = COVID_IO.getLocalDataFrame(MNFIPS, TESTnewJH_state_df)
# Retrieve TEST County-level data for Clay County
TESTnewCLAY_df = COVID_IO.getLocalDataFrame(ClayFIPS, TESTnewJH_cnty_df)

In [ ]:
##
## The Clay county data should have a steady 10 confirmed per day starting March 22, 2020, so the number of confirmed increases as 10 cases per day (means dConfirmed is 10/day)
## First death occurs 14 days later (since I "kill" 10% of the infected at the end of 14 days, the other 90% are recovered).  Check this!

##
## The Minnesota data should have an unreal situation of 10 additional new cases a day from March 22 to June 1.  We still "kill" 10% of the infected 14 days later and mark the
## other 90% "recovered."  Check this.

## Test Apple and Google Mobility DataIO

Execute and test the Apple and Google Mobility DataIO routines first authored by Dio

In [ ]:
# Trying the new routines to import dataframes from CSV with
# conversion of strings to lists.  Also allows a single 
# getLocalDataFrame() function for pulling to local data from
# ANY of our dataframes (since they all index based on FIPS).


## 
## Retrieve the Apple Mobility Data
##

# Retrieve Apple Mobility Dataframe
(newaapl_cnty_df, newaapl_state_df) = COVID_IO.CSVtoAAPLMobilityDataFrames()
# Get real Clay county and Minnesota mobility data
newaapl_CLAY_df = COVID_IO.getLocalDataFrame(ClayFIPS, newaapl_cnty_df)
newaapl_MN_df = COVID_IO.getLocalDataFrame(MNFIPS, newaapl_state_df)

# Retrieve TEST Apple Mobility Dataframe
test_cntyfile = test_dir+"TEST_aapl_mobility_cnty.csv"
test_statefile = test_dir+"TEST_aapl_mobility_state.csv"
(testnewaapl_cnty_df, testnewaapl_state_df) = COVID_IO.CSVtoAAPLMobilityDataFrames(countyFile = test_cntyfile, stateFile = test_statefile)
# Get TEST Clay county and Minnesota mobility data
testnewaapl_CLAY_df = COVID_IO.getLocalDataFrame(ClayFIPS, testaapl_cnty_df)
testnewaapl_MN_df = COVID_IO.getLocalDataFrame(MNFIPS, testaapl_state_df)

## 
## Retrieve the Google Mobility Data
##

# Retrieve Google Mobility Dataframe
(newgoog_cnty_df, newgoog_state_df) = COVID_IO.CSVtoGOOGMobilityDataFrames()
# Get real Clay county and Minnesota mobility data
newgoog_CLAY_df = COVID_IO.getLocalDataFrame(ClayFIPS, newgoog_cnty_df)
newgoog_MN_df = COVID_IO.getLocalDataFrame(MNFIPS, newgoog_state_df)

# Retrieve TEST Google Mobility Dataframe
test_cntyfile = test_dir+"TEST_goog_mobility_cnty.csv"
test_statefile = test_dir+"TEST_goog_mobility_state.csv"
(testnewgoog_cnty_df, testnewgoog_state_df) = COVID_IO.CSVtoGOOGMobilityDataFrames(countyFile = test_cntyfile, stateFile = test_statefile)
# Get TEST Clay county and Minnesota mobility data
testnewgoog_CLAY_df = COVID_IO.getLocalDataFrame(ClayFIPS, testnewgoog_cnty_df)
testnewgoog_MN_df = COVID_IO.getLocalDataFrame(MNFIPS, testnewgoog_state_df)

In [ ]:
##
## Test Apple Mobility Data should be checked
##  - For Clay county, the test data is a sawtooth pattern from -30 to +30 with a 7 day period starting on Feb. 15, 2020
##  - For Minnesota, the test data is a boxcar pattern 4 days at 30 followed by four days at -30 and back again starting on Feb. 15, 2020 starting on Feb. 15, 2020

In [ ]:
##
## Test Google Mobility Data should be checked
##  - For Clay county, the test data is a sine wave with an amplitude of 20 and wavelength of 30 days starting on Feb. 15, 2020
##  - For Minnesota, the test data is a cosine wave with an amplitude of 20 and wavelength of 30 days starting on Feb. 15, 2020

## Test IMHE DataIO

Execute and test the Apple and Google Mobility DataIO routines first authored by Luke

In [ ]:
# Trying the new routines to import dataframes from CSV with
# conversion of strings to lists.  Also allows a single 
# getLocalDataFrame() function for pulling to local data from
# ANY of our dataframes (since they all index based on FIPS).

# Retrieve IMHE Dataframes
(newsummary_df, newhospitalization_df) = COVID_IO.CSVtoIMHEDataFrames()
# Retrieve specific Dataframes and Data for MN
equip_df = COVID_IO.getLocalDataFrame(MNFIPS, newsummary_df)
icu_beds = COVID_IO.GetNumICUBeds(MNFIPS, newsummary_df)
all_beds = COVID_IO.GetNumAllBeds(MNFIPS, newsummary_df)
icu_usage = COVID_IO.GetICUBedUsage(MNFIPS, newsummary_df)
allbed_usage = COVID_IO.GetAllBedUsage(MNFIPS, newsummary_df)
hospital_df = COVID_IO.getLocalDataFrame(MNFIPS, newhospitalization_df)

# Retrieve TEST IMHE Dataframes
testsummaryfile = test_dir+"TEST_imhe_summary.csv"
testhospitalizationfile = test_dir+"TEST_imhe_hospitalizations.csv"
(TESTnewsummary_df, TESTnewhospitalization_df) = COVID_IO.GetIMHEDataFrames(summaryFile = testsummaryfile, hospitalFile = testhospitalizationfile)
# Retrieve specific TEST Dataframes and Data for MN
testnewequip_df = COVID_IO.getLocalDataFrame(MNFIPS, TESTnewsummary_df)
testnewicu_beds = COVID_IO.GetNumICUBeds(MNFIPS, TESTnewsummary_df)
testnewall_beds = COVID_IO.GetNumAllBeds(MNFIPS, TESTnewsummary_df)
testnewicu_usage = COVID_IO.GetICUBedUsage(MNFIPS, TESTnewsummary_df)
testnewallbed_usage = COVID_IO.GetAllBedUsage(MNFIPS, TESTnewsummary_df)
testnewhospital_df = COVID_IO.getLocalDataFrame(MNFIPS, TESTnewhospitalization_df)

In [ ]:
##
## Test IMHE data for Minnesota should be checked.
##  All summary dates were set to May 15, 2020 (lower May 1, upper June 15)
##  Bed capacity was set to 2000 and ICU capacity to 200. 
##  The test data assumes 100% usage.

# The hospitalization data starts on March 22, 2020
# test_imhe_hospitalizations['allbed_mean'] is increasing at 1000 per day (10% margins of error on lower/upper)
# test_imhe_hospitalizations['ICUbed_mean'] is increasing at 100 per day (10% margins of error on lower/upper)
# test_imhe_hospitalizations['InvVen_mean'] is increasing at 50 per day (10% margins of error on lower/upper)
# For the hospitalization deaths/admits/ICU use, the test data has a constant 10 deaths/100 admits/20 ICU a day 
# (with 10% margins of error on lower/upper)